In [3]:
import os
import pdfplumber
import re
import json

def extraire_info(texte, pattern):
    match = re.search(pattern, texte, re.IGNORECASE)
    return match.group(1).strip() if match else None

def traiter_pdf(fichier_pdf):
    with pdfplumber.open(fichier_pdf) as pdf:
        texte = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())

    return {
        "fichier": os.path.basename(fichier_pdf),
        "numero": extraire_info(texte, r"Annonce n°\s*([\d\-]+)"),
        "titre": extraire_info(texte, r"Titre\s*:\s*(.+)"),
        "type_avis": "attribution" if "Avis d’attribution" in texte else "marché",
        "acheteur": extraire_info(texte, r"Nom officiel\s*:\s*(.+)"),
        "forme_juridique": extraire_info(texte, r"Forme juridique.*?:\s*(.+)"),
        "activité": extraire_info(texte, r"Activité.*?:\s*(.+)"),
        "nature": extraire_info(texte, r"Nature du marché\s*:\s*(.+)"),
        "type_procedure": extraire_info(texte, r"Type de procédure\s*:\s*(.+)"),
        "cpv": extraire_info(texte, r"cpv\s*\):\s*(\d{8})"),
        "lieu": extraire_info(texte, r"Subdivision pays.*?:\s*([^\n]+)"),
        "montant_max": extraire_info(texte, r"montant (?:maximum|estimée|quadriennal).*?:\s*([\d\s.,]+)"),
        "duree": extraire_info(texte, r"Durée\s*:\s*([^\n]+)"),
        "date_limite": extraire_info(texte, r"Date limite de réception des offres\s*:\s*(.+)")
    }

def traiter_dossier(dossier, sortie_json):
    fichiers = [os.path.join(dossier, f) for f in os.listdir(dossier) if f.endswith(".pdf")]
    donnees = []

    for f in fichiers:
        try:
            resultat = traiter_pdf(f)
            donnees.append(resultat)
        except Exception as e:
            print(f"Erreur avec {f} : {e}")

    with open(sortie_json, "w", encoding="utf-8") as f_out:
        json.dump(donnees, f_out, ensure_ascii=False, indent=2)

# ✅ Maintenant (si les PDF sont à côté du notebook)
dossier_pdf = "."
fichier_json = "avis_compiles.json"
# ▶️ Lancement
traiter_dossier(dossier_pdf, fichier_json)
